In [1]:
using Plots, ApproxFun, Interact
gr()  # Also works with plotly();

# $$u'' + 2u = \cos(\omega t)\qquad u(0)=u'(0)=0$$

Slide  to $\omega=\sqrt 2$ to see a resonance, where the solution grows

In [2]:
t=Fun(identity,[0.,1000.]) 
L=𝒟^2+2I  # our differential operator, 𝒟 is equivalent to Derivative()

@manipulate for ω=0.:.1sqrt(2):2sqrt(2)
    u=[ivp();L]\[0.;0.;cos(ω*t)]
    plot(u)
end

Interact.Slider{Float64}(Signal{Float64}(1.2727922061357857, nactions=0),"ω",1.2727922061357857,0.0:0.14142135623730953:2.687005768508881,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 25.6518,191.185 25.6523,191.185 25.6532,191.185 25.6546,191.185 25.6565,191.184 25.6588,191.182 25.6617,191.18 25.665,191.175 25.6687,191.169 25.6729,191.16 
 25.6776,191.148 25.6828,191.132 25.6885,191.11 25.6946,191.084 25.7012,191.05 25.7082,191.009 25.7157,190.959 25.7237,190.899 25.7322,190.829 25.7411,190.746 
 25.7505,190.649 25.7604,190.538 25.7707,190.411 25.7815,190.266 25.7928,190.102 25.8046,189.918 25.8168,189.713 25.8295,189.485 25.8426,189.233 25.8563,188.955 
 25.8704,188.652 25.8849,188.321 25.9,187.962 25.9155,187.574 25.9315,187.156 25.9479,186.709 25.9648,186.232 25.9822,185.726 26.0001,185.191 26.0184,184.628 
 26.0372,184.038 26.0565,183.424 26.0762,182.786 26.0964,182.127 26.1171,181.451 26.1382,180.762 26.1598,180.063 26.1819,179.359 26.2045,178.656 26.2275,177.959 
 26.251,177.276 26.2749,176.612 26.2993,175.978 26.3242,175.379 26.3496,174.827 26.3754,174.33 26.4017,173.898 26.4285,173.542 26.4558,173.272 26.4835,173.101 
 26.5116,173.039 26.5403,173.097 26.5694,173.287 26.599,173.62 26.629,174.106 26.6596,174.755 26.6906,175.577 26.722,176.579 26.7539,177.768 26.7863,179.151 
 26.8192,180.73 26.8525,182.507 26.8863,184.482 26.9206,186.652 26.9553,189.012 26.9905,191.552 27.0262,194.261 27.0623,197.125 27.0989,200.124 27.136,203.237 
 27.1736,206.439 27.2116,209.699 27.25,212.986 27.289,216.263 27.3284,219.49 27.3683,222.624 27.4086,225.622 27.4494,228.433 27.4907,231.011 27.5325,233.303 
 27.5747,235.259 27.6174,236.828 27.6605,237.961 27.7041,238.61 27.7482,238.731 27.7928,238.285 27.8378,237.237 27.8833,235.558 27.9292,233.228 27.9756,230.236 
 28.0225,226.579 28.0699,222.267 28.1177,217.319 28.1659,211.768 28.2147,205.661 28.2639,199.057 28.3136,192.027 28.3637,184.658 28.4143,177.049 28.4654,169.31 
 28.5169,161.564 28.5689,153.942 28.6214,146.584 28.6744,139.634 28.7278,133.24 28.7816,127.552 28.8359,122.714 28.8907,118.867 28.946,116.14 29.0017,114.649 
 29.0579,114.495 29.1146,115.755 29.1717,118.484 29.2293,122.708 29.2873,128.424 29.3458,135.595 29.4048,144.15 29.4642,153.98 29.5241,164.942 29.5845,176.857 
 29.6453,189.512 29.7066,202.661 29.7684,216.031 29.8306,229.326 29.8933,242.234 29.9564,254.432 30.02,265.591 30.0841,275.393 30.1487,283.531 30.2136,289.724 
 30.2791,293.723 30.345,295.323 30.4114,294.372 30.4783,290.776 30.5456,284.51 30.6133,275.619 30.6816,264.227 30.7503,250.535 30.8194,234.822 30.889,217.439 
 30.9591,198.811 31.0297,179.419 31.1007,159.796 31.1721,140.511 31.2441,122.153 31.3164,105.315 31.3893,90.5738 31.4626,78.4698 31.5364,69.4874 31.6106,64.0351 
 31.6853,62.4265 31.7604,64.8637 31.836,71.4232 31.9121,82.0458 31.9886,96.5297 32.0656,114.529 32.1431,135.559 32.221,159.002 32.2993,184.125 32.3782,210.099 
 32.4575,236.025 32.5372,260.964 32.6174,283.967 32.6981,304.117 32.7792,320.564 32.8608,332.561 32.9428,339.498 33.0253,340.942 33.1082,336.655 33.1916,326.619 
 33.2755,311.05 33.3598,290.397 33.4446,265.341 33.5299,236.773 33.6155,205.77 33.7017,173.559 33.7883,141.467 33.8754,110.87 33.9629,83.1341 34.0509,59.5521 
 34.1393,41.2828 34.2282,29.2896 34.3176,24.2874 34.4074,26.6965 34.4976,36.6086 34.5883,53.7674 34.6795,77.564 34.7711,107.05 34.8632,140.969 34.9557,177.802 
 35.0487,215.837 35.1422,253.24 35.2361,288.145 35.3304,318.75 35.4252,343.412 35.5205,360.736 35.6162,369.664 35.7124,369.544 35.809,360.178 35.9061,341.858 
 36.0036,315.364 36.1016,281.944 36.2,243.263 36.2989,201.326 36.3983,158.38 36.4981,116.793 36.5983,78.9251 36.699,46.9896 36.8001,22.9212 36.9017,8.25022 
 37.0038,3.99774 37.1063,10.5963 37.2093,27.8432 37.3127,54.8901 37.4165,90.2735 37.5208,131.984 37.6256,177.571 37.7308,224.287 37.8365,269.247 37.9426,309.608 
 38.0491,342.759 38.1561

# Bessel equation $$x^2 u'' + x u + (x^2 - \nu^2)u$$

In [3]:
x=Fun(identity,[1.,2000.])                                 
d=domain(x)
B=dirichlet()  
@manipulate for ν=0:.1:2000
	L=x^2*𝒟^2 + x*𝒟 + (x^2 - ν^2)   # our differential operator
   	u=[B;L]\[besselj(ν,first(d)),besselj(ν,last(d))]
    plot(u;ylims=(-.2,.2))
end

Interact.Slider{Float64}(Signal{Float64}(1000.0, nactions=0),"ν",1000.0,0.0:0.1:2000.0,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,192.476 43.4297,192.476 43.4298,192.476 43.4301,192.476 43.4305,192.476 43.4309,192.476 43.4314,192.476 43.4321,192.476 43.4328,192.476 43.4336,192.476 
 43.4345,192.476 43.4355,192.476 43.4365,192.476 43.4377,192.476 43.439,192.476 43.4403,192.476 43.4417,192.476 43.4433,192.476 43.4449,192.476 43.4466,192.476 
 43.4484,192.476 43.4502,192.476 43.4522,192.476 43.4543,192.476 43.4564,192.476 43.4586,192.476 43.461,192.476 43.4634,192.476 43.4659,192.476 43.4685,192.476 
 43.4712,192.476 43.4739,192.476 43.4768,192.476 43.4797,192.476 43.4828,192.476 43.4859,192.476 43.4891,192.476 43.4924,192.476 43.4958,192.476 43.4993,192.476 
 43.5029,192.476 43.5066,192.476 43.5103,192.476 43.5142,192.476 43.5181,192.476 43.5221,192.476 43.5262,192.476 43.5304,192.476 43.5347,192.476 43.5391,192.476 
 43.5436,192.476 43.5482,192.476 43.5528,192.476 43.5575,192.476 43.5624,192.476 43.5673,192.476 43.5723,192.476 43.5774,192.476 43.5826,192.476 43.5879,192.476 
 43.5932,192.476 43.5987,192.476 43.6042,192.476 43.6099,192.476 43.6156,192.476 43.6214,192.476 43.6273,192.476 43.6333,192.476 43.6394,192.476 43.6455,192.476 
 43.6518,192.476 43.6581,192.476 43.6646,192.476 43.6711,192.476 43.6777,192.476 43.6844,192.476 43.6912,192.476 43.6981,192.476 43.7051,192.476 43.7121,192.476 
 43.7193,192.476 43.7265,192.476 43.7339,192.476 43.7413,192.476 43.7488,192.476 43.7564,192.476 43.7641,192.476 43.7719,192.476 43.7797,192.476 43.7877,192.476 
 43.7957,192.476 43.8039,192.476 43.8121,192.476 43.8204,192.476 43.8288,192.476 43.8373,192.476 43.8459,192.476 43.8546,192.476 43.8633,192.476 43.8722,192.476 
 43.8811,192.476 43.8901,192.476 43.8992,192.476 43.9085,192.476 43.9177,192.476 43.9271,192.476 43.9366,192.476 43.9462,192.476 43.9558,192.476 43.9656,192.476 
 43.9754,192.476 43.9853,192.476 43.9953,192.476 44.0054,192.476 44.0156,192.476 44.0259,192.476 44.0363,192.476 44.0467,192.476 44.0573,192.476 44.0679,192.476 
 44.0786,192.476 44.0894,192.476 44.1003,192.476 44.1113,192.476 44.1224,192.476 44.1336,192.476 44.1448,192.476 44.1562,192.476 44.1676,192.476 44.1792,192.476 
 44.1908,192.476 44.2025,192.476 44.2143,192.476 44.2262,192.476 44.2381,192.476 44.2502,192.476 44.2623,192.476 44.2746,192.476 44.2869,192.476 44.2993,192.476 
 44.3118,192.476 44.3244,192.476 44.3371,192.476 44.3499,192.476 44.3628,192.476 44.3757,192.476 44.3888,192.476 44.4019,192.476 44.4151,192.476 44.4284,192.476 
 44.4418,192.476 44.4553,192.476 44.4689,192.476 44.4826,192.476 44.4963,192.476 44.5102,192.476 44.5241,192.476 44.5381,192.476 44.5522,192.476 44.5664,192.476 
 44.5807,192.476 44.5951,192.476 44.6096,192.476 44.6241,192.476 44.6388,192.476 44.6535,192.476 44.6683,192.476 44.6832,192.476 44.6982,192.476 44.7133,192.476 
 44.7285,192.476 44.7438,192.476 44.7591,192.476 44.7746,192.476 44.7901,192.476 44.8057,192.476 44.8215,192.476 44.8373,192.476 44.8532,192.476 44.8691,192.476 
 44.8852,192.476 44.9014,192.476 44.9176,192.476 44.934,192.476 44.9504,192.476 44.9669,192.476 44.9835,192.476 45.0002,192.476 45.017,192.476 45.0338,192.476 
 45.0508,192.476 45.0679,192.476 45.085,192.476 45.1022,192.476 45.1195,192.476 45.1369,192.476 45.1544,192.476 45.172,192.476 45.1897,192.476 45.2074,192.476 
 45.2253,192.476 45.2432,192.476 45.2613,192.476 45.2794,192.476 45.2976,192.476 45.3159,192.476 45.3343,192.476 45.3527,192.476 45.3713,192.476 45.3899,192.476 
 45.4087,192.476 45.4275,192.476 45.4464,192.476 45.4654,192.476 45.4845,192.476 45.5037,192.476 45.523,192.476 45.5423,192.476 45.5618,192.476 45.5813,192.476 
 45.6009,192.476 45.6206,192.476 45.6404,192.476 45.6603,192.476 45.6803,192.476 45.7004,192.476 45.7205,192.476 45.7408,192.476 45.76

# Exponentially ill-conditioned Lee & Greengard BVP $\epsilon u'' - x u' + u = 0$

In [4]:
x=Fun(identity)
B=dirichlet()

@manipulate for ϵ=0.0005:0.0005:0.4
    L=ϵ*𝒟^2 - x*𝒟 + I

     u=[B;L]\[1.,2.]
    plot(u;ylims=(-2.,3.))
end

Interact.Slider{Float64}(Signal{Float64}(0.2, nactions=0),"ϵ",0.2,0.0005:0.0005:0.4,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 25.6518,154.793 25.769,154.989 26.0032,155.379 26.3545,155.962 26.8227,156.735 27.4075,157.693 28.1088,158.83 28.9262,160.141 29.8595,161.62 30.9081,163.257 
 32.0718,165.044 33.35,166.973 34.7422,169.034 36.2478,171.217 37.8662,173.511 39.5968,175.906 41.4388,178.392 43.3915,180.957 45.454,183.59 47.6256,186.282 
 49.9053,189.021 52.2922,191.798 54.7854,194.602 57.3837,197.423 60.0861,200.254 62.8916,203.083 65.7989,205.905 68.8069,208.709 71.9143,211.49 75.1199,214.24 
 78.4223,216.953 81.8202,219.623 85.3121,222.245 88.8968,224.814 92.5726,227.324 96.3381,229.774 100.192,232.158 104.132,234.474 108.157,236.719 112.265,238.891 
 116.455,240.987 120.725,243.006 125.073,244.947 129.497,246.808 133.996,248.589 138.567,250.289 143.209,251.907 147.92,253.443 152.698,254.897 157.541,256.269 
 162.447,257.558 167.414,258.766 172.439,259.893 177.522,260.938 182.659,261.903 187.85,262.787 193.09,263.592 198.379,264.318 203.714,264.965 209.093,265.535 
 214.514,266.027 219.975,266.443 225.473,266.783 231.006,267.047 236.572,267.237 242.168,267.353 247.793,267.396 253.443,267.366 259.117,267.263 264.812,267.089 
 270.526,266.844 276.257,266.528 282.002,266.142 287.759,265.686 293.525,265.161 299.299,264.568 305.077,263.905 310.857,263.175 316.638,262.377 322.416,261.511 
 328.19,260.578 333.956,259.579 339.713,258.512 345.458,257.379 351.188,256.18 356.903,254.914 362.598,253.583 368.272,252.185 373.922,250.721 379.547,249.191 
 385.143,247.595 390.709,245.934 396.242,244.206 401.74,242.413 407.2,240.553 412.621,238.627 418,236.635 423.336,234.577 428.625,232.453 433.865,230.263 
 439.055,228.006 444.193,225.683 449.275,223.294 454.301,220.838 459.268,218.317 464.174,215.73 469.017,213.078 473.795,210.361 478.506,207.579 483.148,204.733 
 487.719,201.825 492.218,198.855 496.642,195.824 500.99,192.733 505.26,189.585 509.45,186.38 513.558,183.122 517.583,179.813 521.523,176.455 525.377,173.052 
 529.142,169.607 532.818,166.125 536.403,162.608 539.895,159.063 543.293,155.495 546.595,151.909 549.8,148.311 552.908,144.708 555.916,141.108 558.823,137.518 
 561.629,133.947 564.331,130.402 566.929,126.893 569.423,123.43 571.809,120.023 574.089,116.681 576.261,113.415 578.323,110.236 580.276,107.154 582.118,104.182 
 583.849,101.329 585.467,98.6067 586.973,96.0259 588.365,93.5969 589.643,91.3299 590.807,89.2347 591.855,87.3204 592.789,85.5956 593.606,84.0682 594.307,82.7452 
 594.892,81.6328 595.36,80.7365 595.712,80.0604 595.946,79.6078 596.063,79.381 
 "/>
 
 
 
 
 y1

# Piecewise $$u'' + \omega u \begin{cases} 1 & |x| < c\cr 0 & {\rm otherwise}\end{cases}  = 0,\qquad u(-1)=1,u(1)=0$$

In [5]:
@manipulate for ω=1:50000,c=0.1:0.01:1.
    f=Fun(x->abs(x)<c?1:0,[-1.,-c,c,1.])
    S=space(f)
    B=dirichlet(S)
    plot([B;𝒟^2+ω*f]\[1.,0.];ylims=(-0.5,1.1))
end

Interact.Slider{Int64}(Signal{Int64}(25000, nactions=0),"ω",25000,1:50000,"d",true)

Interact.Slider{Float64}(Signal{Float64}(0.55, nactions=0),"c",0.55,0.1:0.01:1.0,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,27.505 43.4323,27.5103 43.4378,27.5207 43.446,27.5364 43.457,27.5574 43.4707,27.5835 43.4872,27.6149 43.5063,27.6515 43.5283,27.6934 43.5529,27.7405 
 43.5804,27.7928 43.6105,27.8503 43.6434,27.913 43.679,27.981 43.7173,28.0541 43.7584,28.1325 43.8022,28.2161 43.8487,28.3049 43.898,28.3988 43.9499,28.498 
 44.0046,28.6024 44.062,28.7119 44.1222,28.8267 44.185,28.9466 44.2505,29.0716 44.3188,29.2019 44.3898,29.3373 44.4634,29.4778 44.5398,29.6235 44.6188,29.7743 
 44.7005,29.9303 44.785,30.0914 44.8721,30.2576 44.9618,30.4289 45.0543,30.6054 45.1494,30.7869 45.2472,30.9735 45.3477,31.1652 45.4508,31.362 45.5566,31.5638 
 45.665,31.7707 45.7761,31.9826 45.8898,32.1996 46.0061,32.4216 46.1251,32.6486 46.2467,32.8806 46.3709,33.1177 46.4977,33.3597 46.6271,33.6066 46.7591,33.8586 
 46.8938,34.1155 47.031,34.3773 47.1708,34.6441 47.3132,34.9157 47.4581,35.1923 47.6056,35.4738 47.7557,35.7602 47.9083,36.0514 48.0635,36.3475 48.2212,36.6485 
 48.3814,36.9542 48.5442,37.2648 48.7095,37.5802 48.8772,37.9004 49.0475,38.2253 49.2203,38.555 49.3956,38.8895 49.5733,39.2287 49.7536,39.5726 49.9362,39.9212 
 50.1214,40.2745 50.309,40.6324 50.499,40.995 50.6915,41.3623 50.8863,41.7342 51.0836,42.1107 51.2833,42.4917 51.4854,42.8774 51.6899,43.2676 51.8968,43.6623 
 52.106,44.0616 52.3176,44.4653 52.5315,44.8736 52.7478,45.2863 52.9664,45.7034 53.1874,46.125 53.4106,46.551 53.6362,46.9814 53.864,47.4162 54.0941,47.8553 
 54.3265,48.2988 54.5612,48.7466 54.7981,49.1987 55.0373,49.655 55.2786,50.1156 55.5222,50.5805 55.768,51.0495 56.0161,51.5228 56.2663,52.0002 56.5186,52.4818 
 56.7732,52.9675 57.0299,53.4573 57.2887,53.9513 57.5497,54.4492 57.8127,54.9513 58.0779,55.4573 58.3452,55.9674 58.6146,56.4814 58.8861,56.9994 59.1596,57.5213 
 59.4351,58.0471 59.7127,58.5768 59.9923,59.1104 60.274,59.6478 60.5576,60.189 60.8432,60.734 61.1308,61.2828 61.4203,61.8354 61.7118,62.3916 62.0053,62.9515 
 62.3006,63.5152 62.5979,64.0824 62.8971,64.6533 63.1981,65.2278 63.5011,65.8058 63.8058,66.3874 64.1125,66.9725 64.4209,67.5611 64.7312,68.1532 65.0433,68.7487 
 65.3571,69.3476 65.6728,69.9499 65.9902,70.5555 66.3093,71.1645 66.6302,71.7769 66.9528,72.3924 67.2771,73.0113 67.6031,73.6334 67.9307,74.2586 68.2601,74.8871 
 68.5911,75.5186 68.9237,76.1533 69.2579,76.7911 69.5937,77.432 69.9312,78.0758 70.2702,78.7227 70.6107,79.3726 70.9528,80.0254 71.2964,80.6811 71.6416,81.3397 
 71.9882,82.0011 72.3363,82.6654 72.6859,83.3325 73.0369,84.0024 73.3894,84.675 73.7433,85.3503 74.0986,86.0283 74.4553,86.7089 74.8134,87.3922 75.1728,88.078 
 75.5336,88.7665 75.8957,89.4574 76.2591,90.1509 76.6238,90.8468 76.9897,91.5451 77.357,92.2459 77.7255,92.9491 78.0952,93.6546 78.4661,94.3624 78.8382,95.0725 
 79.2115,95.7848 79.586,96.4994 79.9616,97.2161 80.3384,97.935 80.7162,98.6561 81.0952,99.3792 81.4752,100.104 81.8563,100.832 82.2385,101.561 82.6217,102.292 
 83.0059,103.025 83.391,103.76 83.7772,104.497 84.1644,105.236 84.5524,105.976 84.9414,106.719 85.3314,107.463 85.7222,108.209 86.1139,108.956 86.5064,109.705 
 86.8998,110.456 87.294,111.208 87.6891,111.962 88.0849,112.717 88.4815,113.474 88.8788,114.232 89.2769,114.992 89.6757,115.753 90.0752,116.515 90.4754,117.279 
 90.8762,118.044 91.2778,118.81 91.6799,119.577 92.0826,120.346 92.486,121.115 92.8899,121.886 93.2944,122.658 93.6994,123.431 94.105,124.205 94.5111,124.98 
 94.9176,125.755 95.3246,126.532 95.7321,127.31 96.14,128.088 96.5483,128.867 96.9571,129.647 97.3662,130.428 97.7757,131.209 98.1855,131.991 98.5956,132.774 
 99.0061,133.557 99.4168,134.341 99.8278,135.125 100.239,135.91 100.651,136.695 101.062,137.481 101.474,138.267 101.886,139.053 102.299,139.84 102.711,140.627 
 103.124,141.414 103.536,1

# Forced Helmholtz equation $$u_{xx} + u_{yy} + k u = {\rm e}^{-(x-x_0)^2-(y-y_0)^2},\qquad \partial u = 0$$

In [37]:
using Interact

myfunction2(k) = begin 
    zeros(Int(1e1)) #takes a few secs on my machine
    2k
end
anotherfunction(g,j) = g*j
k = slider(1:100; label="k")
j = slider(1:100; label="j")
g = map(myfunction2, signal(k)) # g is a signal whose value is myfunction(k)
f = map(anotherfunction, g, signal(j)) # f is a signal whose value is anotherfunction(g,j)
map(display, [k,j]) #display widgets
map(display, [g,f]) #display results;

Interact.Slider{Int64}(Signal{Int64}(50, nactions=1),"k",50,1:100,"d",true)

Interact.Slider{Int64}(Signal{Int64}(50, nactions=1),"j",50,1:100,"d",true)

100

5000

In [1]:
using Interact

k = slider(-100.:.1:100.; label="k")
x₀ = slider(-2.:0.1:2.; label="x₀")
y₀ = slider(-2.:0.1:2.; label="y₀")

f = map((x0,y0)->((x,y)->exp(-(x-x0)^2-(y-y0)^2)), signal(x₀), signal(y₀)) 
p = map(f->f(0.1,0.2),f)

map(display, [k,x₀,y₀]) #display widgets
map(display, p) #display results;

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"k",0.0,-100.0:0.1:100.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"x₀",0.0,-2.0:0.1:2.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"y₀",0.0,-2.0:0.1:2.0,".3f",true)

0.951229424500714

In [ ]:
gr()

d=Interval()^2
B=Dirichlet(d)
Δ=Laplacian(d)
QR = map(k->qrfact([B;Δ+k*I]), signal(k))

In [33]:
gr()

d=Interval()^2
B=Dirichlet(d)
Δ=Laplacian(d)

@manipulate for x0=-1.:0.1:1.,y0=-1.:0.1:1.,k=-200.:0.1:200.
    f=Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2),d)
    u=linsolve([B;Δ+k*I],[0.;f];tolerance=1E-4)
    contour(u)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x0",0.0,-1.0:0.1:1.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"y0",0.0,-1.0:0.1:1.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"k",0.0,-200.0:0.1:200.0,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#f3e258; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 136.969,366.026 135.735,367.088 131.467,369.843 126.352,371.73 125.964,371.872 120.462,373.422 114.96,374.352 109.457,374.907 103.955,375.205 98.4527,375.299 
 92.9503,375.205 87.448,374.907 81.9457,374.352 76.4434,373.422 70.9411,371.872 70.5533,371.73 65.4388,369.843 61.1704,367.088 59.9365,366.026 56.752,362.445 
 54.4363,357.803 54.4342,357.796 52.6292,353.16 51.5653,348.518 50.9607,343.875 50.6846,339.233 50.6846,334.59 50.9607,329.948 51.5653,325.305 52.6292,320.663 
 54.4342,316.027 54.4363,316.021 56.752,311.378 59.9365,307.797 61.1704,306.736 65.4388,303.981 70.5533,302.093 70.9411,301.951 76.4434,300.401 81.9457,299.472 
 87.448,298.916 92.9503,298.618 98.4527,298.524 103.955,298.618 109.457,298.916 114.96,299.472 120.462,300.401 125.964,301.951 126.352,302.093 131.467,303.981 
 135.735,306.736 136.969,307.797 140.153,311.378 142.469,316.021 142.471,316.027 144.276,320.663 145.34,325.305 145.945,329.948 146.221,334.59 146.221,339.233 
 145.945,343.875 145.34,348.518 144.276,353.16 142.471,357.796 142.469,357.803 140.153,362.445 136.969,366.026 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#fabf26; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 132.394,362.445 131.467,363.237 125.964,366.659 124.95,367.088 120.462,368.843 114.96,370.192 109.457,370.994 103.955,371.423 98.4527,371.559 92.9503,371.423 
 87.448,370.994 81.9457,370.192 76.4434,368.843 71.9557,367.088 70.9411,366.659 65.4388,363.237 64.5118,362.445 60.5861,357.803 59.9365,356.636 58.1018,353.16 
 56.5727,348.518 55.7067,343.875 55.3119,339.233 55.3119,334.59 55.7067,329.948 56.5727,325.305 58.1018,320.663 59.9365,317.187 60.5861,316.021 64.5118,311.378 
 65.4388,310.586 70.9411,307.164 71.9557,306.736 76.4434,304.98 81.9457,303.631 87.448,302.829 92.9503,302.4 98.4527,302.264 103.955,302.4 109.457,302.829 
 114.96,303.631 120.462,304.98 124.95,306.736 125.964,307.164 131.467,310.586 132.394,311.378 136.319,316.021 136.969,317.187 138.803,320.663 140.333,325.305 
 141.199,329.948 141.593,334.59 141.593,339.233 141.199,343.875 140.333,348.518 138.803,353.16 136.969,356.636 136.319,357.803 132.394,362.445 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#fb9c07; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 131.467,357.371 131.118,357.803 125.964,362.242 125.636,362.445 120.462,365.111 114.96,366.817 113.621,367.088 109.457,367.972 103.955,368.618 98.4527,368.821 
 92.9503,368.618 87.448,367.972 83.2841,367.088 81.9457,366.817 76.4434,365.111 71.2688,362.445 70.9411,362.242 65.7873,357.803 65.4388,357.371 62.5309,353.16 
 60.6201,348.518 59.9365,345.696 59.4596,343.875 58.8665,339.233 58.8665,334.59 59.4596,329.948 59.9365,328.127 60.6201,325.305 62.5309,320.663 65.4388,316.452 
 65.7873,316.021 70.9411,311.581 71.2688,311.378 76.4434,308.712 81.9457,307.006 83.2841,306.736 87.448,305.851 92.9503,305.205 98.4527,305.002 103.955,305.205 
 109.457,305.851 113.621,306.736 114.96,307.006 120.462,308.712 125.636,311.378 125.964,311.581 131.118,316.021 131.467,316.452 134.374,320.663 136.285,325.305 
 136.969,328.127 137.446,329.948 138.039,334.59 138.039,339.233 137.446,343.875 136.969,345.696 136.285,348.518 134.374,353.16 131.467,357.371 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#f57d15; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 131.467,351.161 130.357,353.16 126.308,357.803 125.964,358.098 120.462,361.713 118.793,362.445 114.96,363.945 109.457,365.298 103.955,366.02 98.4527,366.248 
 92.9503,366.02 87.448,365.298 81.9457,363.945 78.1125,362.445 76.4434,361.713 70.9411,358.098 70.5977,357.803 66.5479,353.16 65.4388,351.161 64.091,348.518 
 62.6486,343.875 61.9926,339.233 61.9926,334.59 62.6486,329.948 64.091,325.305 65.4388,

In [19]:
d=Interval()^2
B=[dirichlet(d[1])⊗I;I⊗ldirichlet(d[2]);I⊗rneumann(d[2])]
Δ=Laplacian(d)

@manipulate for k=-100.:100.,x0=-1.:0.1:1.,y0=-1.:0.1:1.
    f=Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2),d)
    u=linsolve([B;Δ+k*I],[0.;0.;0.;f];tolerance=1E-3,maxlength=2000)
    contour(u)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"k",0.0,-100.0:1.0:100.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x0",0.0,-1.0:0.1:1.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"y0",0.0,-1.0:0.1:1.0,".3f",true)

#     Dirichlet Helmholtz equation $$\Delta u + k u = 0, u(\pm 1,y)=u(x,\pm 1) = 1$$

In [30]:
d=Interval()^2
B=Dirichlet(d)
Δ=Laplacian(d)

@manipulate for k=-200.0:.001:200.0
    contour(linsolve([B;Δ+k*I],[ones(∂(d));0.];tolerance=1E-3))
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"k",0.0,-200.0:0.001:200.0,".3f",true)

# Convection diffusion $u_t = \epsilon u_{xx} + (B+C x) u_x, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t) = 0$

In [41]:
dx=Interval();dt=Interval(0,1.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(dx*dt)
Bcs=timedirichlet(d)
u0=Fun(x->exp(-20x^2),dx)
@manipulate for ε=0.001:0.001:2.,B=-5.:0.1:5.,C=-5.:0.1:5.
    V=B+C*x    
    # Parentheses are a hack to get rank 2 PDE
    u=linsolve([Bcs;Dt-ε*Dx^2-V*Dx],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(u;xlims=(-1.,1.),ylims=(0.,1.))
end

Interact.Slider{Float64}(Signal{Float64}(1.0, nactions=0),"ε",1.0,0.001:0.001:2.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"B",0.0,-5.0:0.1:5.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"C",0.0,-5.0:0.1:5.0,".3f",true)

# Schrodinger equation $$i \epsilon u_t = -{\epsilon^2 \over 2} u_{xx} + x^2 u$$

In [43]:
dx=Interval(0.,1.);dt=Interval(0.0,0.54)
d=dx*dt
x,t=Fun(d)
V=x^2

Dt=Derivative(d,[0,1]);Dx=Derivative(d,[1,0])

@manipulate for ϵ=0.005:0.002:0.3
    u0=Fun(x->exp(-25*(x-0.5)^2)*exp(-1.0im/(5*ϵ)*log(2cosh(5*(x-0.5)))),dx)
    L=1im*ϵ*Dt+0.5*ϵ^2*Dx^2-V

    u=linsolve([timedirichlet(d);L],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(real(u))
end

Interact.Slider{Float64}(Signal{Float64}(0.151, nactions=0),"ϵ",0.151,0.005:0.002:0.299,".3f",true)

Interact.Slider{Int64}(Signal{Int64}(125, nactions=0),"ny",125,50:200,"d",true)

# Nonlinear BVP $$\epsilon u'' + 6(1-x^2)u' +u^2=1$$ $$u(-1)=u(1)=0$$

In [44]:
x=Fun()
u0=0.0x
@manipulate for ε=0.001:0.001:1.,c=0.:0.1:1.
    N=u->[u(-1.)-c;u(1.);ε*u''+6*(1-x^2)*u'+u^2-1.]

    u=newton(N,u0)
    plot(u;ylims=(-1.,1.))
end

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=0),"ε",0.5,0.001:0.001:1.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=0),"c",0.5,0.0:0.1:1.0,".3f",true)